<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_gpt_Sum1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sacrebleu --quiet
!pip install evaluate --quiet
!pip install rouge_score --quiet
!pip install bert_score --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torc

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import openai
import pandas as pd
import numpy as np
import os
import kagglehub
from bert_score import score


from google.colab import userdata

# print(openai.__version__)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_NLP')

In [4]:

import sacrebleu
from nltk.translate.meteor_score import meteor_score
import nltk
import evaluate

# Download necessary resources for METEOR
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but recommended
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# import data from google drive
file_name = 'data_sum1_100'
#'data_sum1_500'


path = "/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/"+file_name+".csv"
data = pd.read_csv(path)
data.head()



,text,summary,Dataset
0,By . Paul Donnelley . The first Range Rover pr...,On launch Land Rover described it as the most ...,CNN/Daily Mail
1,The Shakespeare Birthplace Trust said it had s...,A £5.25m project to transform the site of Shak...,XSum
2,A.P.C. founder Jean Touitou has apologized for...,A.P.C. founder Jean Touitou has apologized fo...,CNN/Daily Mail
3,"The first film, starring Oscar-winning actor E...",Warner Bros has announced a follow-up to upcom...,XSum
4,"By . Mark Duell . PUBLISHED: . 08:14 EST, 16 M...",Tesco unveiled plans to turn a pub in Dorset a...,CNN/Daily Mail


In [ ]:
'''# keep 100 from each Dataset
data_sum1_100 = data.groupby('Dataset').head(100).reset_index(drop=True)
# count for each Dataset
data_sum1_100['Dataset'].value_counts()
'''

"# keep 100 from each Dataset\ndata_sum1_100 = data.groupby('Dataset').head(100).reset_index(drop=True)\n# count for each Dataset\ndata_sum1_100['Dataset'].value_counts()\n"

In [ ]:
#data_sum1_100.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_sum1_100.csv', index=False)

In [7]:
len(data)

300

In [8]:
prompts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_sum1_sample.csv')
prompts.head()

,text,summary,Dataset
0,Former top prosecutor Sir Keir Starmer says ti...,Britain's former top prosecutor says time 'sho...,CNN/Daily Mail
1,"Forget the idea of an old-fashioned ski lodge,...",Club Med: Val Thorens Sensations was designed ...,CNN/Daily Mail
2,Vincent Kompany remains unavailable but is clo...,Manchester City defender Gael Clichy is likely...,XSum
3,"The winning image, by Hadi Asadi from Iran, sh...",Hundreds of cartoonists from around the world ...,XSum
4,\n \n Sam Walton in 1985. (AP Photo) \n \n E...,– Today in the ugly lawsuit department: Walmar...,Multi-News


In [ ]:
prompts['text'][0]

"Former top prosecutor Sir Keir Starmer says time should be a barrier to justice . Britain's former top prosecutor has controversially defended police who hold suspects in a ‘legal limbo’ for years while alleged crimes are investigated. Keir Starmer, the ex- Director of Public Prosecutions, disagreed with those who have called for time limits on how long a person can languish on bail without being charged. He said time ‘should not be a barrier’ to bringing people to justice for crimes committed decades earlier. A number of high-profile celebrities, including Rolf Harris and Stuart Hall, have been convicted of sex crimes that happened years ago after their victims finally plucked up the courage to complain about the attacks. But there have been mounting concerns about cases where suspects are left on bail for long periods – often with strict curbs to their freedom – without charges being pressed. Veteran disc jockey Paul Gambaccini spent 12 months on bail after being arrested by officer

In [ ]:
prompts['summary'][0]

"Britain's former top prosecutor says time 'should not be a barrier' to justice .\nDisagreed with those who call for time limits on how long a person can remain on bail without being charged .\nComes after a number of celebrities, including Rolf Harris and Stuart Hall, have been convicted of crimes committed decades ago ."

In [9]:
zero_shot = f""" """

one_shot = f"""
              Input: {prompts['text'][0]}
              Output: {prompts['summary'][0]}
"""

three_shot = f"""
              Input: {prompts['text'][1]}
              Output: {prompts['summary'][1]}
              Input: {prompts['text'][3]}
              Output: {prompts['summary'][3]}
              Input: {prompts['text'][5]}
              Output: {prompts['summary'][5]}
"""


In [10]:
# text summary with GPT

def summarize_text(text):
    try:
        # Call GPT-3.5 Turbo
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Summarize the following news articles to highlight the key facts. Provide answers to who, what, when, where, why and how in the summary. The summary should be a few sentences long."},
                {"role": "assistant", "content": shot},
                {"role": "user", "content": text}
            ]
        )
        # Extract and return the response
        summary = response.choices[0].message.content.strip()
#       print(response.choices[0].message.content)
        return summary
    except Exception as e:
        return f"Error: {e}"

In [35]:
# adjust for desired prompt / model / data size
test_size = 10 #len(data)   # can adjust for smaller testing

function = summarize_text
shot_name = "three_shot"      # change for zero / one / three shot
model_name = "gpt-3.5-turbo"  # change for different models

In [36]:

prediction = []

if shot_name == "one_shot":
  shot = one_shot
elif shot_name == "three_shot":
  shot = three_shot
else:
  shot = zero_shot



def summarize(data):
  count = 0
  for i, texts in enumerate(data.text[:test_size]):
    result = function(texts)
    count += 1
    if count % 100 == 0:
      print(count)

    prediction.append(result)



In [37]:
summarize(data)


In [38]:
len(prediction)

10

In [39]:
# reference to list
reference = data.summary[:test_size].tolist()


In [40]:
reference[0]

'On launch Land Rover described it as the most versatile car in the world .\nDesigned to combine comfort and on-road ability of Rover saloon with off-road skill of Land Rover .\nStylish two-door model was originally delivered to Michael Forlong, who produced two Range Rover promotion films .\nIt then had a number of owners before vanishing until the 1990s when an enthusiast discovered it .\nHe then spent six years carrying out restoration on the off-roader .\nKnown as #001, it has a 3.5-litre V8 engine and 135bhp .'

In [41]:
prediction[0]

"The first production Range Rover, which set the benchmark for luxury 4x4s, is going up for auction in London on September 4 and is expected to sell for over £150,000. The two-door green model, registered as YVB 151H, was built in 1969 and registered in 1970, designed to combine comfort with off-road capabilities. The car underwent a meticulous restoration and is expected to attract significant interest from enthusiasts and investors at auction. Despite having only two doors, the car is considered a 'matching numbers' model with original components, including a 3.5-liter V8 engine and a top speed of over 100mph."

In [42]:
file_name

'data_sum1_100'

In [43]:
shot_name


'three_shot'

In [44]:
size = len(prediction)
size

10

In [45]:
# view metrics and save
def view_and_save_metrics(prediction, reference):
  # BLEU
  bleu = sacrebleu.corpus_bleu(prediction, [reference])
#  print(f"BLEU: {bleu.score}")

  # Tokenize sentences for METEOR
  prediction_tokens = [nltk.word_tokenize(pred) for pred in prediction]
  reference_tokens = [[nltk.word_tokenize(ref)] for ref in reference]  # Note the double list for reference

  # METEOR (sentence-level average)
  meteor_scores = [
      meteor_score(ref_tokens, pred_tokens)
      for ref_tokens, pred_tokens in zip(reference_tokens, prediction_tokens)
  ]
  avg_meteor = sum(meteor_scores) / len(meteor_scores)

  # ROUGE
  rouge = evaluate.load('rouge')
  rouge_scores = rouge.compute(predictions=prediction, references=reference)
#  print(f"ROUGE: {rouge_scores}")

  # BERT Score
  P_, R_, F1 = score(prediction, reference, lang="el")
  # print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
  bert_score = F1.mean().item()

  metrics = {
      "bleu": bleu.score,
      "meteor": avg_meteor,
      "bert_score": bert_score,
      "model_name": model_name,
      "dataset": file_name,
      "shot": shot_name,
      "size": size
  }

  metrics.update(rouge_scores)
  '''
  # Print all metrics
  print("\n--- Evaluation Metrics ---")
  print(f"BLEU:   {bleu.score:.4f}")
  print(f"METEOR: {avg_meteor:.4f}")
  for k, v in rouge_scores.items():
      print(f"{k.upper()}: {v:.4f}")
  '''

  # save metrics to csv file
  metrics_df = pd.DataFrame([metrics])
  metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/metrics/summarization/metrics_'+model_name+'_'+file_name+'_'+shot_name+'_'+str(size)+'.csv', index=False)
  return metrics_df

In [46]:
metrics = view_and_save_metrics(prediction, reference)
metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,4.776009,0.291851,0.702336,gpt-3.5-turbo,data_sum1_100,three_shot,10,0.278971,0.086445,0.179653,0.235603
